In [46]:
import os
import matplotlib.pyplot as plt
import numpy as np
import warnings
import configparser
warnings.filterwarnings('ignore')

import matplotlib.cm as cm

import pykoopman as pk

In [47]:
pk.__version__

'1.0.8'

In [48]:
config = configparser.ConfigParser()
config.optionxform = str
config.read("config.cfg")
args = dict(config["BLOCKED_10"])
args

{'data_path': 'data/Xfull_10_blocked.npy',
 'th': '0.010517',
 'N_x': '7',
 'N_u': '4',
 'N_d': '0',
 'maxN_T': '120',
 'buffer': '10'}

In [49]:
## Initialize/load data and params

Xfull = np.load(args["data_path"])
Xfull = Xfull[:, 50:, :]


N_x, N_u, N_d = int(args["N_x"]), int(args["N_u"]), int(args["N_d"])

X0 = Xfull[:N_x, :, :]
X1 = Xfull[:-N_x, :, :]
U = Xfull[: N_x + N_u, :, :]

print(Xfull.shape)

n_T, N_T = 30, min(Xfull.shape[1], int(args["maxN_T"]))  # Number of sample trajectories, Length of ea. Trajectory
th = float(args["th"])*int(args["buffer"])
t0, tf = 0.0, th * (Xfull.shape[1])
t_arr = np.arange(t0, tf, th)[:N_T]
print(Xfull.shape, t_arr.shape)


(18, 141, 1)
(18, 141, 1) (120,)


In [50]:
N_T

120

In [51]:
train_p = 0.6
idxs = np.arange(Xfull.shape[2])
np.random.shuffle(idxs)
# x_shuffled = x[idxs] how do i fix this line

# num_train_samples = int(Xfull.shape[2] * (train_p))
num_train_samples = 1
X_train_i = idxs[:num_train_samples]
X_test_i = idxs[num_train_samples:]

In [52]:
def plot_comparison(model, train_ix, no_comp=False, EDMDc=True, N_T=20):

    n_columns = 4  # Define the number of columns you want
    n_rows = 2 # (n_T * 4 + n_columns - 1) // n_columns  # Calculate the number of rows needed

    plt.figure(figsize=(15 * n_columns, 8 * n_rows))
    t_arr = np.arange(t0, tf, th)[:N_T]

    for i in range(1):
        X = Xfull[:, :N_T, i]

        if np.isnan(X[:N_x, 0].T).any() or np.isnan(X[N_x : N_x + N_u, :].T).any():
            print(X[:N_x, 0].T.shape)
            print(X[N_x : N_x + N_u, :].T.shape)
            print("nan")
            return

        axs = []; labels = ["X", "Y", "Z", "dX", "dY", "dZ", "dTh", "Err"]
        for j in range(N_x + 1):
            axj = plt.subplot(n_rows, n_columns, (i * 4 + j) % (n_rows * n_columns) + 1)
            if j < N_x:
                axj.plot(t_arr, X[j, :], alpha=0.8, label="Nominal", color="blue")
            axj.set_xlabel("t"); axj.set_ylabel(labels[j])
            axs.append(axj)

        if not (no_comp):
            if not (EDMDc):
                Xhp = model.simulate(
                    X[:N_x, 0].T, X[N_x : N_x + N_u, :].T, n_steps=N_T - 1
                )
                Xh = np.vstack((X[:N_x, 0].T, Xhp)).T
            else:
                print(np.isnan(X[:N_x, 0]).any())
                Xh = model.simulate(
                    X[:N_x, 0].reshape(1, -1), X[N_x : N_x + N_u, :].T, n_steps=N_T
                    # X[:N_x, 0].reshape(1, -1), n_steps=10
                ).T

        for j in range(N_x):
            axs[j].plot(t_arr, Xh[j, :], alpha=0.8, label="Model", color="red")

        axs[-1].plot(t_arr, np.linalg.norm(X[:N_x, :] - Xh, axis=0), alpha=0.8, label="Error", color="pink")

        # # Adjust this if you have additional logic for highlighting non-training data
        # for ax in [ax1, ax2, ax3, ax4]:
        #     for spine in ax.spines.values():
        #         spine.set_linewidth(2.0)  # Increase the width of the plot spines
        #         if i not in train_ix:
        #             spine.set_edgecolor("red")

    if no_comp:
        plt.suptitle("Crazyflie Trajectories", fontsize=50)
    else:
        plt.suptitle("EDMDc Linearizations of Crazyflie Trajectories", fontsize=50)
    plt.show()

    # print(Xh.shape)
    return Xh

In [53]:
# plot_comparison(None, [], no_comp=True)

In [54]:
def fitK(X_full, train_ix, regressor, N_T=100, svd_rank=None, method="EDMDc"):
    Xb = X_full[:, :N_T, train_ix]

    ## Flatten Data (only matters if training with more than one traj)
    Xks, Xkps, Uks = [], [], []
    print(len(train_ix))

    for i in range(len(train_ix)):
        Xks.append(Xb[:N_x, :, i])
        Xkps.append(Xb[N_x + N_u :, :, i])
        Uks.append(Xb[N_x : N_x + N_u, :, i])
    Xk, Xkp, Uk = np.hstack(Xks).T, np.hstack(Xkps).T, np.hstack(Uks).T
    print(Xk.shape, Xkp.shape, Uk.shape)
    print(np.isnan(Xk).any())
    # DMDc = pk.regression.DMDc(svd_rank=svd_rank)
    EDMDc = pk.regression.EDMDc()
    # EDMD = pk.regression.EDMD()
    DMDc = pk.regression.DMDc(svd_rank=svd_rank)
    # model = pk.Koopman(observables=observables, regressor=EDMDc).fit(Xk, u=Uk, y=Xkp)
    # model = pk.Koopman(observables=observables, regressor=EDMD).fit(Xk, y=Xkp)

    print("Xkp:", Xkp.shape)
    model = pk.Koopman(regressor=regressor).fit(Xk, y=Xkp)

    return model

In [55]:
import lightning as L
class IncrementalSequenceLoss(L.Callback):
    def on_train_epoch_end(self, trainer, pl_module):
        max_look_forward = pl_module.look_forward
        if trainer.callback_metrics["loss"] < 1e-2 and \
                pl_module.masked_loss_metric.max_look_forward < max_look_forward:
            print("increase width size from {} to {}".format(
                pl_module.masked_loss_metric.max_look_forward,
                pl_module.masked_loss_metric.max_look_forward+1) )
            print("")
            pl_module.masked_loss_metric.max_look_forward += 1

In [56]:
Xk, Uk = Xfull[:N_x, :].T, Xfull[N_x : N_x + N_u, :-1].T

look_forward = 50
dlk_regressor = pk.regression.NNDMD(dt=nonlinear_sys.dt, look_forward=look_forward,
                                    config_encoder=dict(input_size=2,
                                                        hidden_sizes=[32] * 3,
                                                        output_size=3,
                                                        activations="swish"),
                                    config_decoder=dict(input_size=3, hidden_sizes=[],
                                                        output_size=2, activations="linear"),
                                    batch_size=512, lbfgs=True,
                                    normalize=True, normalize_mode='equal',
                                    normalize_std_factor=1.0,
                                    trainer_kwargs=dict(max_epochs=3))

model = fitK(X_full=Xfull, train_ix=X_train_i, regressor=dlk_regressor, svd_rank=-1)
file_name = "Blocked_10_obs_RBF10_reg_EDMDc.npy"
# np.save(f"models/{file_name}", model, allow_pickle=True)

if np.isnan(model.A.any()):
    print("NAN")
## Compare True and DMDc
Xh = plot_comparison(model, train_ix=X_train_i, N_T=N_T)
# Xh[0, :]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/site-packages/lightning/pytorch/trainer/configuration_validator.py:69: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
/usr/local/lib/python3.11/site-packages/pykoopman/regression/_nndmd.py:876: UserWarning: Warning: no validation data prepared
  warn("Warning: no validation data prepared")

  | Name                | Type                    | Params
----------------------------------------------------------------
0 | _encoder            | FFNN                    | 18.6 K
1 | _decoder            | FFNN                    | 13.1 K
2 | _koopman_propagator | StandardKoopmanOperator | 64    
3 | masked_loss_metric  | MaskedMSELoss           | 0     
-----------------------

1
(100, 7) (100, 7) (100, 4)
False
Xkp: (100, 7)
Epoch 0:   0%|          | 0/10 [00:00<?, ?it/s] 

RuntimeError: The expanded size of the tensor (7) must match the existing size (4) at non-singleton dimension 1.  Target sizes: [10, 7].  Tensor sizes: [10, 4]